# ネットワークモデルの実装、順伝播関数の実装
SSDのネットワークモデルと順伝播forward関数を実装

## 目標
### ネットワークモデル
1. SSDのネットワークを構築している４つのモジュールを把握
2. SSDのネットワークモデルを作成できるようになる
3. SSDで使用する様々な大きさのDafaultBoxの実装方法を理解する

### 順伝播
1. Non-Maximum Suppressionを理解
2. SSDの推論時に使用するDetectクラスの順伝播を理解する
3. SSDの順伝播を実装できるようにする

## Library

In [16]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Function
import torch.nn.functional as F

import numpy as np
import pandas as pd

from math import sqrt
from itertools import product

## vggモジュールを実装

In [10]:
# 34層に渡るvggモジュールの作成
# 各層の設定をリスト(cfg)で保持してfor文回してlayerを作っていく

def make_vgg():
    layers = []
    in_channels = 3   # 色チャネル情報
    
    # vggモジュールで使用する畳み込みそうやマックスプーリングのチャネル数
    cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'MC', 512, 512, 512, 'M', 512, 512, 512]
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v == 'MC':
            # ceilは出力shapeの計算で、計算結果(float)に対して、切り上げ整数にするモード
            # defaultでは出力サイズを計算結果に対して切り下げて整数にするfloorモードになっている
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            # pooling以外であればconv層
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v     # 次のconv用に更新しておく
            
    # 最後だけ変則的な設定なのでfor文の中に入れていない      
    pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
    conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
    conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
    layers += [pool5, conv6, nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    
    return nn.ModuleList(layers)


# 動作確認
vgg_test = make_vgg()
print(vgg_test)
    
# OK  34層のvggネットワークができている        

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

In [11]:
# リストにリストを足していくとどうなるか  これを上のlayerでやっている
l1 = []
tmp_list = []
for i in range(5):
    tmp_list = [i, i, i]
    l1 += tmp_list
print(l1)

# リストにリストをappendしていくとどうなるか
l1 = []
tmp_list = []
for i in range(5):
    tmp_list = [i, i, i]
    l1.append(tmp_list)
print(l1)

# リストにリストをextendしていくとどうなるか
l1 = []
tmp_list = []
for i in range(5):
    tmp_list = [i, i, i]
    l1.extend(tmp_list)
print(l1)

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]
[[0, 0, 0], [1, 1, 1], [2, 2, 2], [3, 3, 3], [4, 4, 4]]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]


## extraモジュールの実装

In [13]:
# 8層のextraモジュールを作成
# source2を入力として、source3-6を出力する　書くsourceは特徴マップの大きさが違う
# 活性化関数のReLUはssdモデルのforwardで用意することにして、extraモジュールでは実装しない

def make_extras():
    layers = []
    in_channels = 1024   # vggモジュールから出力されたsource2の画像チャネル数
    
    # extraモジュールの畳み込み層のチャネル数を設定するconfig
    cfg = [256, 512, 128, 256, 128, 256, 128, 256]
    
    # 細かい設定が異なるのでfor文でまとめられない
    layers += [nn.Conv2d(in_channels, cfg[0], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[0], cfg[1], kernel_size=(3), stride=2, padding=1)]
    layers += [nn.Conv2d(cfg[1], cfg[2], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[2], cfg[3], kernel_size=(3), stride=2, padding=1)]
    layers += [nn.Conv2d(cfg[3], cfg[4], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[4], cfg[5], kernel_size=(3))]
    layers += [nn.Conv2d(cfg[5], cfg[6], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[6], cfg[7], kernel_size=(3))]
    
    return nn.ModuleList(layers)

# 動作確認
extras_test = make_extras()
print(extras_test)

# OK  

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)


## locモジュールとconfモジュールの実装

In [14]:
# default boxのoffsetを出力するloc_layers
# default boxに対する各クラスの信頼度confidenceを出力するconf_layersを作成
# どちらのモジュールもsource1-6に対し各々convを噛ませている
# ２０分類＋背景１の２１つのクラスに分類する

def make_loc_conf(num_classes=21, bbox_aspect_num=[4, 6, 6, 6, 4, 4, 4]):
    """
    num_classes: int
        分類するクラス数（物体２０分類＋背景１分類）
    
    bbox_aspect_num: source1-6に対するdefault boxの種類の数
    """
    
    loc_layers = []
    conf_layers = []
    
    # vggの２２層目、conv4_3(source1)に対する畳み込み層
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[0]*4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[0]*num_classes, kernel_size=3, padding=1)]
    
    # vggの最終層(source2)に対する畳み込み層
    loc_layers += [nn.Conv2d(1024, bbox_aspect_num[1]*4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(1024, bbox_aspect_num[1]*num_classes, kernel_size=3, padding=1)]
    
    # extraの(source3)に対する畳み込み層
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[2]*4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[2]*num_classes, kernel_size=3, padding=1)]
    
    # extraの(source4)に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[3]*4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[3]*num_classes, kernel_size=3, padding=1)]
    
    # extraの(source5)に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[4]*4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[4]*num_classes, kernel_size=3, padding=1)]
    
    # extraの(source6)に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[5]*4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[5]*num_classes, kernel_size=3, padding=1)]
    
    return nn.ModuleList(loc_layers), nn.ModuleList(conf_layers)


# 動作確認
loc_test, conf_test = make_loc_conf()
print(loc_test)
print(conf_test)

#  OK  出力したい変数分のoutputになっている


ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


## L2Norm層の実装

In [25]:
# convC4_3からの出力をscale=20のL2Normで正規化する層
class L2Norm(nn.Module):
    def __init__(self, input_channels=512, scale=20):
        super(L2Norm, self).__init__()   #  親クラスのコンストラクタ実行
        self.weight = nn.Parameter(torch.Tensor(input_channels))
        self.scale = scale                # 係数weightの初期値として設定する値
        self.reset_parameters()   # parameterの初期化
        self.eps = 1e-10
        
    def reset_parameters(self):
        """
        結合パラメータを大きさscaleの値にする初期化を実行
        """
        init.constant_(self.weight, self.scale)    # weightの値が全てscaleになる
        
    def forward(self, x):
        """
        38x38の特徴量に対して５１２チャネルに渡って二乗和のルートを求めた38x38この値を使用し、
        各特徴量を正規化してから係数を掛け算する層
        """
        # 各チャネルにおける３８ｘ３８個の特徴量のチャネル方向の二乗和を計算し、ルートを求め割り算して正規化する
        # normのテンソルサイズはtorch.Size([batch_num, 1, 38, 38])になる
        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt() + self.eps  # 0除算を防ぐためのeps
        x = torch.div(x, norm)
        
        # 係数をかける。係数を書くチャネルごとに一つで512この係数をもつ
        # self.weightのテンソルサイズはtorch.Size([512])なので
        # torch.Size([batch_num, 512, 38, 38])まで変形する
        weights = self.weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)  # unsqueeze : 次元を増やす
        out = weights * x
        
        return out
        

## default boxの実装

In [22]:
# default boxを出力するクラス
# cfgに結構いろんな設定をしておくようである
class DBox(object):
    def __init__(self, cfg):
        super(DBox, self).__init__()    # 親クラスのコンストラクタを実行するときにクラスの名前入れなくてもいいはず
        
        # 初期設定
        self.image_size = cfg['input_size']  # 画像サイズの300
        # [38, 19, ...]各sourceの特徴マップのサイズ
        self.feature_maps = cfg['feature_maps']
        self.num_priors = len(cfg['feature_maps'])   # source の個数＝６
        self.steps = cfg['steps']  # [8, 16, ...] DBoxのピクセルサイズ
        self.min_sizes = cfg['min_sizes']  # [30, 60, ...] 小さい方の正方形のDBoxのピクセルサイズ
        self.max_sizes = cfg['max_sizes']  # [60, 111, ...] 大きい方の正方形のDBoxのピクセルサイズ
        self.aspect_ratios = cfg['aspect_ratios']  # 長方形のDBoxのアスペクト比
        
    def make_dbox_list(self):
        """
        DBoxを作成する
        """
        mean = []
        # 'feature_maps' : [38, 19, 10, 5, 3, 1]
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):  # fまで数で2ペアの組み合わせを作る f_P_2個
                # 特徴量の画像サイズ
                # 300 / 'steps' : [8, 16, 32, 64, 100, 300],
                f_k = self.image_size / self.steps[k]
                
                # DBoxの中心座標x, y ただし。０−１で規格化している
                cx = (j+0.5) / f_k
                cy = (i+0.5) / f_k
                
                # アスペクト比１の小さいDBox[cx, cy, width, height]
                # 'min_sizes' : [30, 60, 111, 162, 213, 264]
                s_k = self.min_sizes[k] / self.image_size
                mean += [cx, cy, s_k, s_k]
                
                # アスペクト比１の大きいDBox [cx, cy, width, height]
                # 'max_sizes' : [60, 111, 162, 213, 264, 315],
                s_k_prime = sqrt(s_k*(self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]
                
                # その他のアスペクト比のdefbox [cx, cy, width, height]
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
                
        # DBoxをlist -> tensorに変換 torch.Size([8732, 4])
        output = torch.Tensor(mean).view(-1, 4)
        
        # DBoxが画像の外にはみ出るのを防ぐため、大きさを最小０，最大１にする
        output.clamp_(max=1, min=0)
        
        return output

In [23]:
# DBoxの動作の確認

# ssd300の設定
ssd_cfg = {
    'num_classes' : 21,                                                                   #　背景クラスを含めた合計クラス数
    'input_size' : 300,                                                                     # 画像の入力サイズ
    'bbox_aspect_num' : [4, 6, 6, 6, 4, 4],                                  # 出力するDBoxのアスペクト比の種類
    'feature_maps' : [38, 19, 10, 5, 3, 1],                                   # 各source（feature map）の画像サイズ　
    'steps' : [8, 16, 32, 64, 100, 300],                                        # DBOXの大きさを決める
    'min_sizes' : [30, 60, 111, 162, 213, 264],                         # 
    'max_sizes' : [60, 111, 162, 213, 264, 315],
    'aspect_ratios' : [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# DBox作成
dbox = DBox(ssd_cfg)
dbox_list = dbox.make_dbox_list()

# DBoxの出力を確認する
pd.DataFrame(dbox_list.numpy())

,0,1,2,3
0,0.013333,0.013333,0.100000,0.100000
1,0.013333,0.013333,0.141421,0.141421
2,0.013333,0.013333,0.141421,0.070711
3,0.013333,0.013333,0.070711,0.141421
4,0.040000,0.013333,0.100000,0.100000
...,...,...,...,...
8727,0.833333,0.833333,0.502046,1.000000
8728,0.500000,0.500000,0.880000,0.880000
8729,0.500000,0.500000,0.961249,0.961249
8730,0.500000,0.500000,1.000000,0.622254


## SSDクラスの実装
ここまで作成したモジュールをまとめてssdクラスを実装する
pytorchのネットワーク層クラスのnn.Moduleを継承


In [26]:
# ssdクラスを作成
class SSD(nn.Module):
    
    def __init__(self, phase, cfg):
        super(SSD, self).__init__()  # 親クラスのコンストラクタを実行
        
        self.phase = phase   # 'train' or 'inference' を指定
        self.num_classes = cfg['num_classes']
        
        # ssdのネットワークを作る
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(
            cfg['num_classes'], cfg['bbox_aspect_num'])
        
        # DBox作成
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        # 推論時はDetectクラスを用意
        if phase == 'inference':
            self.detect = Detect()

# 動作確認
ssd_test = SSD(phase='train', cfg=ssd_cfg)
print(ssd_test)

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

## 順伝播関数の実装


## 目標

### 順伝播
1. Non-Maximum Suppressionを理解
2. SSDの推論時に使用するDetectクラスの順伝播を理解する
3. SSDの順伝播を実装できるようにする

## decode関数を実装
dbox = (cx_d, cy_d, w_d, h_d)と、loc =(Δcx, Δcy, Δw, Δh)を使用してbboxの座標を取得する関数

In [40]:
def decode(loc, dbox_list):
    """
    オフセット情報を使ってDBoxをBBoxに変換する
    
    Parameters
    ----------------
    loc: [8732, 4]
        ssdモデルで推論したオフセット情報
    dbox_list: [8732, 4]
        DBoxの情報
        
    Returns
    ----------------
    boxes: [xmin, ymin, xmax, ymax]
        BBoxの情報
    """
    
    # DBoxは[cx,, cy, width, height]で格納されている
    # locも[Δcx, Δcy, Δw, Δh]で格納されている
    
    # オフセット情報からBBoxを求める
    boxes = torch.cat((
        dbox_list[:, :2]+0.1*loc[:, :2]*dbox_list[:, 2:],
        dbos_list[:, 2:]*torch.exp(0.2*loc[:, 2:])), dim=1)      # 2列ずつ計算して最後cat, dim=1で横につなげる
        
    # boxesのサイズは[8732, 4]
    # bboxの座標を[cx, cy, width, height] -> [xmin, ymin, xmax, ymax]に変換
    boxes[:, :2] -= boxes[:, 2:] / 2   # xmin, yminに変換
    boxes[:, 2:] += boxes[:, :2]        # xmax, ymaxに変換
    
    return boxes

In [30]:
# torch.catのデモ
input1 = torch.randn(2,3,4)
input2 = torch.randn(2,3,4)
input3 = torch.randn(2,3,4)
input_list = [input1, input2, input3]

output1 =  torch.cat(input_list, dim=0)
print(output1.size())
output2 =  torch.cat(input_list, dim=1)
print(output2.size())
output3 =  torch.cat(input_list, dim=2)
print(output3.size())

torch.Size([6, 3, 4])
torch.Size([2, 9, 4])
torch.Size([2, 3, 12])


In [33]:
# わざとエラーになるようにしてあるセル
input1 = torch.randn(2, 3, 4)
input2 = torch.randn(2, 3, 4)
input3 = torch.randn(5, 3, 4)
input_list = [input1, input2, input3]


output1 = torch.cat(input_list, dim=0)
print(output1.size())
output2 = torch.cat(input_list, dim=1)  # error  dim=0のshapeがあっていないから
print(output2.size()) 
output3 = torch.cat(input_list, dim=2)　　# error  dim=0のshapeがあっていないから
print(output3.size()) 

torch.Size([9, 3, 4])


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 2 and 5 in dimension 0 at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/TH/generic/THTensor.cpp:612

## Non-Maximum Supressionを行う関数を実装
同じ物体に対する複数のBBoxの重複を確信度が最も高いもののみを残し、他のBBoxは消去する操作 = Non-Maximum Supression

In [71]:
def nm_supression(boxes, scores, overlap=0.45, top_k=200):
    """
    Non-Maximum Supressionを行う関数
    boxesの内被りすぎ（overlap以上）のBBoxを削除する
    
    Parameters
    ----------------
    boxes : [確信度閾値（０．０１）を超えたBBox数, 4]
        4は[xmin, ymin, xmax, ymax]の４つ
        BBox情報
    scores :  [確信度閾値（０．０１）を超えたBBox数]
        conf情報
    overlap : float
        BBoxがかぶっているかどうかを判定する閾値
    top_k : 
        
    Returns
    ----------------
    keep : list
        confの降順にnmsを通過したindexが格納
    count : int
        nmsを通過したBBoxの数
    """
    # scoresの型確認
    print(type(scores))
    
    # return の雛形を作成
    count = 0
    keep = scores.new(scores.size(0)).zero_().long()
    
    # keep : torch.Size([確信度閾値を超えたBBox数])、要素は全部０
    
    # 各BBoxの面積areaを計算
    x1 = boxes[:, 0]    # xmin
    y1 = boxes[:, 1]    # ymin
    x2 = boxes[:, 2]    # xmax
    y2 = boxes[:, 3]    # ymax
    area = torch.mul(x2-x1, y2-y1)
    
    # boxesをコピーする。後でBBoxのかぶり度合いIOUの計算に使用する際の雛形(template)として使う
    tmp_x1 = boxes.new()
    tmp_y1 = boxes.new()
    tmp_x2 = boxes.new()
    tmp_y2 = boxes.new()
    tmp_w = boxes.new()
    tmp_h = boxes.new()
    
    # scoreを昇順に並び替える
    v, idx = scores.sort(0)
    
    # 上位top_k（２００）個のBBoxのindexを取り出す
    # 200個なかったときに out of rangeにならないのだろうか　ならなかった
    idx = idx[-top_k:]
    
    # idxの要素が0でない限りループする
    while idx.numel() > 0:
        i = idx[-1]  # 現在のconfの最大のindexをiにする
        
        # keepの現在の最後にconfの最大のindexを格納する
        # このindexのBBoxと被りが大きいBBoxを以降で消去する
        keep[count] = i
        count += 1
        
        # 最後のループになったときはループを抜ける
        if idx.size(0) == 1:
            break
        
        # 現在のconf最大のindexをkeepに格納したいのでidxを一つ減らす
        idx = idx[:-1]
        
        # -----------------
        # これからkeepに格納したBBoxと被りの大きいBBoxを抽出して除去する
        # -----------------
        # 一つ減らしたidxまでのBBoxを、outに指定した変数として作成する
        torch.index_select(x1, 0, idx, out=tmp_x1)
        torch.index_select(y1, 0, idx, out=tmp_y1)
        torch.index_select(x2, 0, idx, out=tmp_x2)
        torch.index_select(y2, 0, idx, out=tmp_y2)
        
        # 全てのBBOxに対して、現在のBBox=indexがiと被って居る値までに設定（clamp）
        tmp_x1 = torch.clamp(tmp_x1, min=x1[i])
        tmp_y1 = torch.clamp(tmp_y1, min=y1[i])
        tmp_x2 = torch.clamp(tmp_x2, min=x2[i])
        tmp_y2 = torch.clamp(tmp_y2, min=y2[i])
        
        # wとhのテンソルサイズをindexを一つ減らしたものにする
        tmp_w.resize_as_(tmp_x2)
        tmp_h.resize_as_(tmp_y2)
        
        # clampした状態でのBBoxの幅と高さを求める
        tmp_w = tmp_x2 - tmp_x1
        tmp_h = tmp_y2 - tmp_y1
        
        # 幅や高さが負になっているものは0にする
        tmp_w = torch.clamp(tmp_w, min=0.0)
        tmp_h = torch.clamp(tmp_h, min=0.0)
        
        # clampされた状態での面積を求める
        inter = tmp_w * tmp_h
        
        # IoU = intersect部分　/ (area(a) + area(b) - intersect部分)の計算
        # BBox同士の論理積　/ BBox同士の論理和
        rem_areas = torch.index_select(area, 0, idx)   # 各BBoxの元の面積
        union = (rem_areas - inter) + area[i]  # 2つのエリアの和(OR)の面積
        IoU = inter / union
        
        # IoUがoverlapより小さいidxのみ残す
        idx = idx[IoU.le(overlap)]   # le : Less than equal to
        
    # whileのループが抜けたら終了
    
    return keep, count

In [41]:
# torch.new()の確認
a = torch.new((3, 2)).zero_()
# tensor.new()らしい

AttributeError: module 'torch' has no attribute 'new'

In [50]:
# torch.numel()の確認
# 入力したtensorの全要素数を返す
a = torch.randn(4, 6)
print(a)

out = torch.numel(a)
print(out)

tensor([[-1.1850,  0.2771,  0.7181, -0.6247,  0.1221, -0.2196],
        [ 0.4402, -0.3734, -0.7193, -0.7542,  0.3715, -0.4340],
        [ 0.7251,  0.5971,  0.6088,  0.9716,  0.9566,  1.2273],
        [ 1.5245,  2.1797, -0.4631, -1.1362, -1.7648,  0.1946]])
24


In [45]:
# torch.mulの確認
a = torch.arange(1,5,1)
b = torch.arange(1,5,1)
print(a*b)
print(torch.mul(a, b))

tensor([ 1,  4,  9, 16])
tensor([ 1,  4,  9, 16])


In [61]:
# out of rangeにならないか確認
a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]  # len = 16
a = a[-20:]
print(a)
b = a[:20]
print(b)
# 後ろからでも前からでもならなかった

# 降順にしたいときはステップで指定する　 うまくいかなかった
a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
a = a[-20::-1]
print(a)
b = a[:20:-1]
print(b)

# うまくいった
a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
a = a[:-20:-1]
print(a)
a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
b = a[20::-1]
print(b)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[]
[]
[16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
[16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


In [ ]:
# torch.clampの確認
# 最大値、最小値を指定の値で抑える

In [ ]:
# torch.index_selectの確認

## Detectクラスの実装
推論時の最後にDetectクラスを使って（batch_num, 21, 200, 5）の出力テンソルを作成する

In [79]:
class Detect(Function):
    
    def __init__(self, conf_thresh=0.01, top_k=200, nms_thresh=0.45):
        self.softmax = nn.Softmax(dim=-1)  # confをソフトマックス関数で正規化するため
        self.conf_thresh = conf_thresh         # confがconf_thresh=0.01より高いDBoxのみをあつかう
        self.top_k = top_k                               # nm_supressionでconfの高いtop_k個を計算に使用する
        self.nms_thresh = nms_thresh          # nm_supressionでIoUがnms_thresh=0.45より大きいと同一物体へのBBoxとみなす
        
    def forward(self, loc_data, conf_data, dbox_list):
        """
        順伝播の計算を実行する
        
        Parameters
        ----------------
        loc_data : [batch_num, 8732, 4]
            オフセット情報
        conf_data : [batch_num, 8732, num_classes]
            検出の確信度
        dbox_list : [8732, 4]
            DBoxの情報
        
        Returns
        ----------------
        output : torch.Size([batch_num, 21, 200, 5])
            [batch_num, クラス, confのtop200, BBoxの情報]     なんで５なんだろう
        """
        
        # 各サイズを取得
        num_batch = loc_data.size(0)
        num_dbox = loc_data.size(1)
        num_classes = conf_data(2)
        
        # confはsoftmaxをかまして正規化
        conf_data = self.softmax(conf_data)
        
        # 出力の型を作成。テンソルサイズは[mini_batch数、21, 200, 5]
        # 一旦０埋めで形だけ作ってしまう
        output = torch.zeros(num_batch, num_classes, self.top_k, 5)
        
        # conf_dataを [batch_num, 8732, num_classes] -> [batch_num, num_classes, 8732]
        conf_preds = conf_data.transpose(2,1)
        
        # ミニバッチごとのループ
        for i in range(num_batch):
                
                # 1. loc（オフセット情報）とdbox_list(Default Boxの形と座標をまとめたもの)から修正したBBox[xmin, ymin, xmax, ymax]を求める
                decoded_boxes = decode(loc_data[i], dbox_list)
                
                # confのコピーを作成  どうしてcloneするのか
                conf_scores = conf_preds[i].clone     # [num_classes, 8732]になっている
                
                # 画像クラスごとのループ（背景クラスのindexである０は計算せず、index=1から）
                for cl in range(1, num_classes):
                    
                    # 2.confの閾値を超えたBBoxを取り出す
                    # confの閾値を超えているかのマスクを作成し閾値を超えたconfのインデックスをc_maskとして取得
                    c_mask = conf_scores[cl].gt(self.conf_thresh)       # gt : greater than  でconf_threshよりおおきいconfは１、小さいconfは０になる
                    # conf_scores : torch.Size([num_classes=21, 8732])
                    # c_mask : torch.Size([8732])
                    
                    # scoresはtorch.Size([閾値を越えたBBox数])
                    scores = conf_scores[cl][c_mask]
                    
                    # 閾値を超えたconfがない場合、つまりscores= []のときは何もしない
                    if scores.nelement() == 0:
                        continue
                    
                    # c_maskをdecoded_boxesに適応できるようにサイズを変更する
                    l_mask = c_mask.unsqeeze(1).expand_as(decoded_boxes)   # 次元を増やしてdecoded_boxesの形に拡張する
                    # l_mask : torch.Size([8732, 4])
                    
                    # l_maskをdecoded_boxesに適応
                    boxes = decoded_boxes[l_mask].view(-1, 4)
                    # よくわかんないけどdecoded_boxes[l_mask]で１次元になってしまうのでviewで[閾値を超えたBBox数, 4]に変形し直す
                    
                    # 3. Non-Maximum Suppressionを実施し、被っているBBoxを取り除く
                    ids, count = nm_suppression(boxes, scores, self.nms_thresh, self.top_k)
                    # ids : confの降順にNon-Maximum Suppressionを通過したindexが格納されている
                    # count : Non-Maximum Suppressionを通過したBBoxの数
                    
                    # outputにNon-Maximum Suppressionを抜けた結果を格納
                    # output : torch.SIze([num_batch, num_classes, self.top_k, 5])
                    output[i, cl, :count] = torch.cat((scores[ids[:count]].unsquueeze(1), boxes[ids[:count]], 1))  # 横にconcat
                    
        return output  # torch.size([1, 21, 200, 5])   なんで１なんだ？
                    

In [78]:
# nelementの確認  （torch.numelのエイリアス）
a = torch.randn(1,2,3,4,5)
print(a.nelement())
print(a.numel())
print(a.shape)

b = torch.zeros(4,4)
print(b.nelement())
print(b.numel())

120
120
torch.Size([1, 2, 3, 4, 5])
16
16


## SSDモデルの実装 2回目
ここまで実装したものをまとめて、forwardまで作る

In [ ]:
# ssdクラスを作成

class SSD(nn.Module):
    
    def __init__(self, phase, cfg):
        super(self).__init__()   # ここわざとSSD(クラス名)入れないでやってみる
        
        self.phase = phase   # 'train' or 'inference'
        self.num_classes = cfg['num_classes']  # クラス数=21
        
        # SSDのネットワークを作る
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(cfg['num_classes'], cfg['bbox_aspect_num'])
        
        # DBox作成
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        # inferenceのときはクラスDetectを用意する
        if phase == 'inference':
            self.detect = Detect()
    
    def forward(self, x):
        sources = list()   # locとconfへの入力　source1-6を格納
        loc = list()
        conf = list()
        
        # vggのconv4_3まで計算  make_vggで作ったモデルの(23)の出力
        for k in range(23):
            x = self.vgg[k](x)
            
        # conv4_3の　出力をL２Normに入力しsorce1を作成、sourcesに追加
        source1 = self.L2Norm(x)
        sources.append(source1)
        
        # vggを最後まで計算しsource2を作成、sourcesに追加
        for k in range(23, len(self.vgg)):
            x = self.vgg[k](x)
   
        sources.append(x)
        
        # extrasのconvとReLUを計算
        # source3-6をsourcesに追加
        for k, v in enumerate(self.extras):
            # 1conv毎にreluを挟む
            x = F.relu(v(x), inplace=True)
            # 2conv毎にsourceとして抽出してsourcesに追加
            if k%2 == 1:
                sources.append(x)
                
        # source1-6にそれぞれ対応する畳込みを１回ずつ適用する
        # zipでforループ回して、複数のリストの要素を一つずつ取得
        # source1-6まであるので６回ループが回る
        

 (extras): ModuleList(
    (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
    (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  )